Syntax natürlicher Sprachen, WS 2023/24

# 08 - Aufgabenblatt

In [1]:
import nltk
from nltk.tree import Tree
from nltk.featstruct import FeatStruct
from nltk.featstruct import Feature, UnificationFailure, FeatStructReader
import itertools

In [2]:
TYPE = nltk.featstruct.TYPE

def check_sanity_constraints(th):
    for type1, type2 in itertools.product(th, th):
        if type1 in th[type2] and type2 in th[type1]:
            if type1 != type2:
                raise ValueError(
                    "The type hierarchy is not antisymmetric! " +
                    "{} subsumes {} and vice versa!".format(
                        type1, type2
                    )
                )

def refl_trans_closure(type_hierarchy):
    # make everything a set
    # and compute reflexive closure
    closure = {}
    for t in type_hierarchy:
        closure[t] = set(type_hierarchy[t])
        closure[t].add(t)

    # compute transitive closure
    still_changes = True
    while still_changes:
        still_changes = False
        for x in closure:
            new_for_x = set()
            for y in closure[x]:
                for z in closure[y]:
                    new_for_x.add(z)
            len_before = len(closure[x])
            closure[x].update(new_for_x)
            still_changes |= len(closure[x]) > len_before

    return closure

class HierarchicalFeature(Feature):
    def __init__(self, name, type_hierarchy, **kwargs):
        super(HierarchicalFeature, self).__init__(name, **kwargs)

        self.hierarchy = refl_trans_closure(type_hierarchy)
        check_sanity_constraints(self.hierarchy)

    def unify_base_values(self, fval1, fval2, bindings):
        candidates = self.hierarchy[fval1].intersection(self.hierarchy[fval2])
        score = {t: 0 for t in candidates}
        for type1, type2 in itertools.product(candidates, candidates):
            if type1 in self.hierarchy[type2]:
                score[type1] += 1

        return min(candidates, key=score.__getitem__, default=UnificationFailure)

## Aufgabe 1 - Kasusrektion, Agreement und Subkategorisierung

### Welche der folgenden morphosyntaktischen Beschränkungen ist in den unteren Sätzen jeweils verletzt?

1. **Kasusrektion**
2. **Agreement** (nominale oder verbale Kongruenz)
3. **Subkategorisierung** (Anzahl und Art der verbalen Argumente)

(i) *Das Hund läuft schnell.*

In [ ]:
#

(ii)  *Den Hund läuft schnell.*

In [ ]:
#

(iii)  *Die Hunde läuft schnell.*

In [ ]:
#

(iv)  *Der Hund glaubt schnell.*

In [ ]:
#

## Aufgabe 2 - Unifikation und Subsumption


### a) Stehen die beiden folgenden Merkmalstrukturen in einer Subsumptions- und/oder Unifikationsbeziehung? Wenn ja, geben Sie das Ergebnis der Subsumption bzw. Unifikation an, erläutern Sie andernfalls:


In [129]:
f0 = FeatStruct("[NUM=pl, GEN=neutr]") 
f1 = FeatStruct("[AGR=[NUM=pl, GEN=neutr]]")

In [ ]:
#Subsumption (f0 ⊑ f1)
f0.subsumes(f1)

In [ ]:
#Subsumption (f1 ⊑ f0)
f1.subsumes(f0)

In [ ]:
#Unification (f0 ⊔ f1)
print(f0.unify(f1))

### b) Stehen die beiden folgenden Merkmalstrukturen in einer Subsumptions- und/oder Unifikationsbeziehung? Wenn ja, geben Sie das Ergebnis der Subsumption bzw. Unifikation an, erläutern Sie andernfalls:

In [133]:
f0 = FeatStruct("[NUM=pl, GEN=neutr]") 
f1 = FeatStruct("[]")

In [ ]:
#Subsumption (f0 ⊑ f1)
f0.subsumes(f1)

In [ ]:
#Subsumption (f1 ⊑ f0)
f1.subsumes(f0)

In [ ]:
#Unification (f0 ⊔ f1)
print(f0.unify(f1))

## Aufgabe 3 - Unifikation

#### Gegeben seien folgende Merkmalstrukturen:

In [3]:
f1 = FeatStruct(
    '[Vorname=Max, Nachname=Mustermann,' + 
    'Privat=[Strasse=Hauptstrasse, Ort=[Stadt=Muenchen]]]'
)
f2 = FeatStruct(
    '[Arbeit=[Strasse="Oettingenstrasse", Ort=(1)[Stadt=Muenchen]],' +
    'Privat=[Ort->(1)]]')
f3 = FeatStruct(
    '[Strasse="Hauptstrasse"]'
)
f4 = FeatStruct(
    '[Privat=[Strasse="Hauptstrasse", Ort=[Stadt=Passau]]]'
)

### a) Unifizieren f1 und f4? Gegeben Sie ggf. das Ergebnis an.

In [4]:
print(f1)

[ Nachname = 'Mustermann'                         ]
[                                                 ]
[            [ Ort     = [ Stadt = 'Muenchen' ] ] ]
[ Privat   = [                                  ] ]
[            [ Strasse = 'Hauptstrasse'         ] ]
[                                                 ]
[ Vorname  = 'Max'                                ]


In [5]:
print(f4)

[          [ Ort     = [ Stadt = 'Passau' ] ] ]
[ Privat = [                                ] ]
[          [ Strasse = 'Hauptstrasse'       ] ]


In [ ]:
#

In [ ]:
#### Mit der Ausführung des print-Statements können Sie abschließend die Lösung einsehen:
print(f1.unify(f4))

### b) Unifizieren f2 und f3? Gegeben Sie ggf. das Ergebnis an.


In [6]:
print(f2)

[          [ Ort     = (1) [ Stadt = 'Muenchen' ] ] ]
[ Arbeit = [                                      ] ]
[          [ Strasse = 'Oettingenstrasse'         ] ]
[                                                   ]
[ Privat = [ Ort -> (1) ]                           ]


In [7]:
print(f3)

[ Strasse = 'Hauptstrasse' ]


In [ ]:
# 

In [ ]:
#### Mit der Ausführung des print-Statements können Sie abschließend die Lösung einsehen:
print(f2.unify(f3))

## Aufgabe 4 - Typhierarchie

#### Gegeben sei folgende Typhierarchie:

$$\bot \sqsubseteq \text{Singular}$$
$$\bot \sqsubseteq \text{nonSingular}$$
$$\text{nonSingular} \sqsubseteq \text{Plural}$$
$$\text{nonSingular} \sqsubseteq \text{Dual}$$


### a) Implementieren Sie ein Feature `NUMBER`, das der vorgegebenen Typhierarchie entspricht.

#### Hier muss die Typhierarchie in Form eines Dictionary definiert werden:

In [8]:
type_hierarchy = {
    
}

In [9]:
NUMBER = HierarchicalFeature("NUMBER", type_hierarchy)
reader = FeatStructReader(features=(NUMBER,))

### b) Geben Sie eine (nicht-leere) Merkmalstruktur `f1` aus der Typhierarchie an, sodass gilt:

`f1` subsumiert `f0`.

In [ ]:
f0 = reader.fromstring("[*NUMBER*=Dual]")
f1 = reader.fromstring("[]")
f1.subsumes(f0)

In [ ]:
f0.subsumes(f1)

In [ ]:
print(f0.unify(f1))

### c) Geben Sie eine (nicht-leere) Merkmalstruktur `f1` aus der Typhierarchie an, sodass gilt:

`f1` subsumiert NICHT `f0`.

In [ ]:
f0 = reader.fromstring("[*NUMBER*=Dual]")
f1 = reader.fromstring("[]")
f1.subsumes(f0)

In [ ]:
f0.subsumes(f1)

In [ ]:
print(f0.unify(f1)) #unvereinbare Merkmale

### d) Geben Sie eine (nicht-leere) Merkmalstruktur `f1` aus der Typhierarchie an, sodass gilt:

`f1` und `f0` unifizieren. 

#### Überlegen Sie auch, ob `f1` und `f0` in einer Subsumptionsbeziehung stehen können.

In [ ]:
f0 = reader.fromstring("[PERS=1]")
f1 = reader.fromstring("[]")
print(f1.unify(f0))

In [ ]:
f0.subsumes(f1)

In [ ]:
f1.subsumes(f0)